In [13]:
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from langchain_community.utilities import GoogleSerperAPIWrapper
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain.agents import Tool
from autogen_core import AgentId,MessageContext,RoutedAgent,message_handler
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost,GrpcWorkerAgentRuntime
from dataclasses import dataclass
import os
from dotenv import load_dotenv

In [14]:
load_dotenv(override=True)
gemini_model  = "gemini-2.0-flash"
gemini_api_key = os.getenv("GEMINI_API_KEY")
gemini_client = OpenAIChatCompletionClient(model=gemini_model,api_key=gemini_api_key)

In [15]:
@dataclass
class Message():
    content : str

In [16]:
serp = GoogleSerperAPIWrapper()
lang_tool = Tool(name="serper",func=serp.run,description="useful tool for google searches")
auto_gen_tool =LangChainToolAdapter(lang_tool)
tools = [auto_gen_tool]

In [17]:
class Player1(RoutedAgent):
    def __init__(self,name:str)->None:
        super().__init__(name)
        self._delegate = AssistantAgent(name,model_client=gemini_client,tools=tools,reflect_on_tool_use=True)
   
    @message_handler
    async def handle_message(self,message:Message,ctx:MessageContext)->Message:
        text_message = TextMessage(content=message.content,source="user")
        response = await self._delegate.on_messages([text_message],cancellation_token=ctx.cancellation_token)
        return Message(content=response.chat_message.content)



In [18]:
class Player2(RoutedAgent):
    def __init__(self,name:str)->None:
        super().__init__(name)
        self._delegate = AssistantAgent(name,model_client=gemini_client,tools=tools,reflect_on_tool_use=True)

    @message_handler
    async def handle_message(self,message:Message,ctx:MessageContext)->Message:
        text_message = TextMessage(content=message.content,source="user")
        response = await self._delegate.on_messages([text_message],cancellation_token=ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [19]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

In [20]:
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        
        self._delegate = AssistantAgent(name, model_client=gemini_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)

In [21]:
ALL_IN_ONE_WORKER = False
host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start()

In [22]:
if ALL_IN_ONE_WORKER:
    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Player1.register(worker,"player1",lambda:Player1("player1"))
    await Player2.register(worker,"player2",lambda:Player2("player2"))
    await Judge.register(worker,"judge",lambda:Judge("judge"))
    agent_id = AgentId("judge","default")
else:
    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1.register(worker1,"player1",lambda:Player1("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2.register(worker2,"player2",lambda:Player2("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker,"judge",lambda:Judge("judge"))
    agent_id = AgentId("judge","default")
    


In [23]:
from IPython.display import display,Markdown
response = await worker.send_message(Message(content="GO!"),agent_id)
display(Markdown(response.content))

## Pros of AutoGen:
Here are some reasons to choose AutoGen for your new AI Agent project, highlighting its pros:

*   **Complex Automation & Dynamic Environments:** AutoGen excels in scenarios requiring complex automation and adaptation to dynamic environments. It's suitable for tasks that go beyond simple, pre-defined workflows.
*   **Multi-Agent Collaboration:** AutoGen is designed for orchestrating multiple AI agents that can collaborate to solve problems. This allows for a division of labor and more sophisticated problem-solving approaches.
*   **Customization:** The framework offers a high degree of customization, allowing you to tailor the agents and their interactions to your specific needs.
*   **Reduced Coordination Complexity:** Natural language handoffs between agents simplify inter-agent communication, eliminating the need for custom protocols and streamlining the development process.
*   **Memory and Learning:** AutoGen supports persistent memory, enabling agents to recall past tasks, user preferences, and learn over time.
*   **LLM Optimization:** AutoGen enables you to efficiently utilize multiple large language models (LLMs) and tools.
*   **Open-Source and Flexible:** Being open-source, AutoGen provides flexibility and community support.

TERMINATE


## Cons of AutoGen:
Here are some cons to consider when deciding whether to use AutoGen for your AI Agent project:

*   **Overkill for Simple Tasks:** AutoGen is designed for complex collaborative tasks. If your project involves simple, straightforward tasks, AutoGen might be more complex than necessary.
*   **Steeper Learning Curve:** The flexibility and customization options can lead to a steeper learning curve compared to simpler frameworks. Understanding how to configure and manage multiple agents and their interactions requires more effort.
*   **Debugging Complexity:** When multiple agents interact, debugging issues can become complex. Tracing the flow of information and identifying the source of errors across different agents can be challenging.
*   **Resource Intensive:** Managing and running multiple collaborating agents can be more resource-intensive than single-agent systems, potentially requiring more powerful hardware and computational resources.
*   **Lack of Visual Builder/No-Code Interface:** AutoGen doesn't offer visual builders or no-code interfaces, making it less accessible to users without strong coding skills.
*   **Complexity in Prompt Engineering:** Effective collaboration between agents relies heavily on well-designed prompts. Crafting prompts that ensure agents understand their roles, communicate effectively, and avoid conflicts can be intricate.
*   **Potential for Unexpected Behavior:** The dynamic interaction between multiple agents can sometimes lead to unexpected or undesirable behavior, especially if the agents' goals are not perfectly aligned or their communication is not carefully managed.

TERMINATE




## Decision:

Based on the provided pros and cons, I will proceed with using AutoGen for the project if and only if the project requires complex automation, multi-agent collaboration, and dynamic adaptation. The key deciding factors are the need for these advanced features, balanced against the increased complexity in learning, debugging, and resource requirements. If the project involves only simple tasks, AutoGen is likely overkill. If the project needs to leverage multiple LLMs and tools and agents need to learn and remember context AutoGen may also be more useful than other frameworks.
TERMINATE


In [24]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()
await host.stop()